## FLAML (A Fast Library for Automated Machine Learning & Tuning) demos

In [1]:
!pip3 install --upgrade flaml catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 206 kB 5.3 MB/s 
     |████████████████████████████████| 76.6 MB 2.2 MB/s 
     |████████████████████████████████| 2.0 MB 40.5 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


### Classification

In [2]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                     header=None, names=('sepal length', 'sepal width', 'petal length', 'petal width', 'species'))
df

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df.species = le.fit_transform(df.species)
labels = le.classes_

labels

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

x_train = train.drop(['species'], axis=1)
y_train = train.species
x_test = test.drop(['species'], axis=1)
y_test = test.species

In [5]:
from flaml import AutoML

clf = AutoML()
clf.fit(x_train, y_train, task='classification', time_budget=60)

[flaml.automl: 09-10 12:58:37] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-10 12:58:37] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-10 12:58:37] {2605} INFO - Evaluation method: cv
INFO:flaml.automl:Evaluation method: cv
[flaml.automl: 09-10 12:58:37] {2727} INFO - Minimizing error metric: log_loss
INFO:flaml.automl:Minimizing error metric: log_loss
[flaml.automl: 09-10 12:58:37] {2869} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 09-10 12:58:37] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-10 12:58:37] {3308} INFO - Estimated sufficient time budget=1065s. Estimated necessary time bu

In [6]:
print('Best ML leaner:', clf.best_estimator)
print('Best hyperparmeter config:', clf.best_config)

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 4, 'max_leaves': 8, 'min_child_weight': 1.9769544888359256, 'learning_rate': 0.7810714944571435, 'subsample': 0.9217787712382914, 'colsample_bylevel': 0.8677415296092055, 'colsample_bytree': 0.9075031859714534, 'reg_alpha': 0.00391652455182368, 'reg_lambda': 0.09655989453029154}


In [7]:
predicted = clf.predict(x_test)

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted, target_names=labels))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30



### Regression

In [9]:
import pandas as pd

df = pd.read_csv('https://github.com/PacktPublishing/Automated-Machine-Learning-with-AutoKeras/raw/main/boston.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,6.48,22.0


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
from flaml import AutoML

reg = AutoML()
reg.fit(dataframe=train, label='MEDV', task='regression', 
        estimator_list=['lgbm', 'xgboost', 'catboost'], 
        time_budget=300)

[flaml.automl: 09-10 12:59:37] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-10 12:59:37] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-10 12:59:37] {2605} INFO - Evaluation method: cv
INFO:flaml.automl:Evaluation method: cv
[flaml.automl: 09-10 12:59:37] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-10 12:59:37] {2869} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'catboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'catboost']
[flaml.automl: 09-10 12:59:37] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-10 12:59:38] {3308} INFO - Estimated sufficient time budget=1100s. Estimated necessary time budget=2s.
INFO:flaml.automl:Estimated sufficient time budget=1100s. Estimated necessary time budget=2s.
[flaml.auto

In [12]:
print('Best ML leaner:', reg.best_estimator)
print('Best hyperparmeter config:', reg.best_config)

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.07050966893761848, 'n_estimators': 90}


In [13]:
x_test = test.drop('MEDV', axis=1)
y_test = test.MEDV

predicted = reg.predict(x_test)

In [14]:
from sklearn.metrics import r2_score, mean_absolute_error

print('Prection R2:', r2_score(y_test, predicted).round(3))
print('Prection MAE:', mean_absolute_error(y_test, predicted).round(3))

Prection R2: 0.859
Prection MAE: 2.019
